In [1]:
Nx = 300
Ny = 300
NxNy = Nx * Ny

dx = 0.03
dy = 0.03

0.03

Time integration parameters

In [2]:
nstep = 4000
nprint = 50;

In [3]:
dtime = 1.e-4

0.0001

Material specific parameters


In [20]:
tau = 0.0003
epsilonb = 0.01
mu = 1.0
kappa = 1.8
delta = 0.02
aniso = 4.0
alpha = 0.9
gamma = 10.0
teq = 1.0
theta0 = 0.2
seed = 5.0

pix = 4.0 * atan(1.0)

3.141592653589793

In [10]:
function nucleus(Nx, Ny, seed)
    phi = zeros(Nx,Ny)
    tempr = zeros(Nx,Ny)

    for i = 1 : Nx
        for j = 1 : Ny
        if ((i-Nx/2)*(i-Nx/2) + (j - Ny/2)*(j-Ny/2) < seed)
            phi[i,j] = 1.0
        end
        end
    end
    return phi, tempr
end

nucleus (generic function with 1 method)

Initialize and introduce the initial nuclei:

In [11]:
phi, tempr = nucleus(Nx,Ny, seed)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [22]:
using NBInclude
@nbinclude("write_vtk_file.ipynb")
using Printf
lap_phi = zeros(Nx,Ny)
lap_tempr = zeros(Nx,Ny)
phidx = zeros(Nx,Ny)
phidy = zeros(Nx,Ny)
epsilon = zeros(Nx,Ny)
epsilon_deriv = zeros(Nx,Ny)
for istep = 1:nstep
    for i = 1 : Nx
        for j = 1 : Ny
            
            jp = j + 1
            jm = j - 1
            ip = i + 1
            im = i - 1
            
            if (im == 0)
                im = Nx
            end
            if (ip == (Nx + 1))
                ip = 1
            end
            
            if (jp == (Ny + 1))
                jp = 1
            end
            if (jm == 0)
                jm = Ny
            end
            
            hne = phi[ip,j]
            hnw = phi[im,j]
            hns = phi[i,jm]
            hnn = phi[i,jp]
            hnc = phi[i,j]
            
            lap_phi[i,j] = (hnw + hne + hns + hnn -4.0*hnc)/(dx*dx)
            
            hne = tempr[ip,j]
            hnw = tempr[im,j]
            hns = tempr[i,jm]
            hnn = tempr[i,jp]
            hnc = tempr[i,j]
            
            lap_tempr[i,j] = (hnw + hne + hns + hnn -4.0*hnc)/(dx*dx)
            
            phidx[i,j] = (phi[ip,j]-phi[im,j])/dx
            phidy[i,j] = (phi[i,jp]-phi[i,jm])/dy
            
            # calculate angle
            theta = atan(phidy[i,j], phidx[i,j])
            
            # epsilon and its derivative:
            epsilon_deriv[i,j] = -epsilonb * aniso * delta * sin(aniso*(theta-theta0))
            epsilon[i,j] = -epsilonb * (1.0 + delta*cos(aniso*(theta-theta0)))
        end
    end
    
    
    for i = 1 : Nx
        for j = 1 : Ny
            jp = j + 1
            jm = j - 1
            
            ip = i + 1
            im = i - 1
            
            if (im == 0)
                im = Nx
            end
            if (ip == Nx + 1)
                ip = 1
            end
            if (jm == 0)
                jm = Ny
            end
            if (jp == Ny + 1)
                jp = 1
            end
            
            phiold = phi[i,j]
            
            # first term
            term1 = (epsilon[i,jp] * epsilon_deriv[i,jp] * phidx[i,jp] - epsilon[i,jm] * epsilon_deriv[i,jm]*phidx[i,jm])/dy
            # second term
            term2 = -(epsilon[ip,j] * epsilon_deriv[ip,j]*phidy[ip,j] - epsilon[im,j] * epsilon_deriv[im,j]*phidy[im,j])/dy
            
            # factor m
            m = alpha/pi*atan(gamma*(teq-tempr[i,j]))
            
            # time integration
            phi[i,j] = phi[i,j]+(dtime/tau)*(term1 + term2 + epsilon[i,j]^2 * lap_phi[i,j] + 
                       phiold*(1.0-phiold)*(phiold-0.5+m))
            
            # evolve temperature
            tempr[i,j] = tempr[i,j] + dtime *lap_tempr[i,j] + kappa*(phi[i,j]-phiold)
        end
    end
    
    if (istep % nprint == 0)
        @printf("done step %5d\n",istep)
        # write vtk file
        write_vtk_grid_values(Nx,Ny,dx,dy,istep,phi);
    end  
end
            
            
            

done step    50
done step   100
done step   150
done step   200
done step   250
done step   300
done step   350
done step   400
done step   450
done step   500
done step   550
done step   600
done step   650
done step   700
done step   750
done step   800
done step   850
done step   900
done step   950
done step  1000
done step  1050
done step  1100
done step  1150
done step  1200
done step  1250
done step  1300
done step  1350
done step  1400
done step  1450
done step  1500
done step  1550
done step  1600
done step  1650
done step  1700
done step  1750
done step  1800
done step  1850
done step  1900
done step  1950
done step  2000
done step  2050
done step  2100
done step  2150
done step  2200
done step  2250
done step  2300
done step  2350
done step  2400
done step  2450
done step  2500
done step  2550
done step  2600
done step  2650
done step  2700
done step  2750
done step  2800
done step  2850
done step  2900
done step  2950
done step  3000
done step  3050
done step  3100
done ste